# Init
Requires GPU to use quantization

In [1]:
!pip install -q -U transformers==4.37.2
!pip install -q bitsandbytes==0.41.3 accelerate==0.25.0

In [ ]:
!pip install pillow

#Video to Frames

In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

Disk version. Saves frames to a folder for human examination/clustering, etc. We can open the images later, if necessary.

In [3]:
import cv2
import math

video_path = "/under_water.mp4"
output_path = "/media" #ATTENTION! The contents of this folder will be removed before each run.

def addzeros(i, length):
  ans = str(i)
  while(len(ans)<len(str(length))):
    ans = "0"+ ans
  return ans

import os
import glob

files = glob.glob(output_path+"/*")
for f in files:
    os.remove(f)

num_frames = 6 #desired number of frames

vid_obj = cv2.VideoCapture(video_path)

length = int(vid_obj.get(cv2.CAP_PROP_FRAME_COUNT))
print(length)

frames = []

i = 0
result = True

while(i<length):
    vid_obj.set(cv2.CAP_PROP_POS_FRAMES,i)
    result, image = vid_obj.read()
    frames.append(image)
    output_fullpath = output_path + "/frame" + addzeros(i,length) + ".jpg"
    cv2.imwrite(output_fullpath,image)
    i+=math.ceil(length/num_frames)


2437


RAM version. A slightly faster version for direct evaluation with BLIP. No files are stored.

In [4]:
import cv2
import math

video_path = "/under_water.mp4"

num_frames = 6 #desired number of frames

vid_obj = cv2.VideoCapture(video_path)

length = int(vid_obj.get(cv2.CAP_PROP_FRAME_COUNT))
print(length)

frames = []

i = 0
result = True

while(i<length):
    vid_obj.set(cv2.CAP_PROP_POS_FRAMES,i)
    result, image = vid_obj.read()
    frames.append(image)
    i+=math.ceil(length/num_frames)

2437


# Blip

In [5]:
import os
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# list to store files
res = []

descriptions = []
for raw_image in frames:
    inputs = processor(raw_image, return_tensors="pt")

    out = model.generate(**inputs)
    descriptions.append(processor.decode(out[0], skip_special_tokens=True))

print(descriptions)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['a fish tank with a fish inside of it', 'a boat in a tank with fish swimming around', 'a large fish swimming in a tank filled with water', 'a large group of fish swimming in an aquarium', 'a large group of fish swimming in a tank', 'a large group of fish swimming in a tank']
